In [1]:
import os
os.chdir("..")

In [2]:
print(os.path.abspath(os.curdir))

e:\Data Kuliah\S2 ITS\Semester 4\Seminar 2\Definition Extraction\drone-definition


In [14]:
import csv

# Define the columns in the CSV file
WORD_COLUMN = 'words'
ACTUAL_LABEL_COLUMN = 'actual_class'
PREDICTED_LABEL_COLUMN = 'predicted_class'

# Read the CSV file
filename = 'ner_results/bert-base-cased/prediction_bert-base-cased.csv'

actual_spans = []
predicted_spans = []

with open(filename, 'r') as file:
    reader = csv.DictReader(file)
    current_actual_span = []
    current_predicted_span = []
    for row in reader:
        word = row[WORD_COLUMN]
        actual_label = row[ACTUAL_LABEL_COLUMN]
        predicted_label = row[PREDICTED_LABEL_COLUMN]

        # Check if it's the beginning of a new entity span
        if actual_label.startswith('B-'):
            if current_actual_span:
                actual_spans.append(current_actual_span)
                predicted_spans.append(current_predicted_span)
                current_actual_span = []
                current_predicted_span = []

        # Check if it's the continuation of an entity span or a non-entity token
        if actual_label.startswith(('B-', 'I-')):
            current_actual_span.append((word, actual_label[2:]))
            current_predicted_span.append((word, predicted_label[2:]))

    # Append the last span if it exists
    if current_actual_span:
        actual_spans.append(current_actual_span)
        predicted_spans.append(current_predicted_span)

# Print the constructed entity spans
# print("Actual Spans:")
# for span in actual_spans:
#     print(span)
# print()
# print("Predicted Spans:")
# for span in predicted_spans:
#     print(span)

# Calculate TP, FP, and FN
tp = 0
fp = 0
fn = 0

for actual_span, predicted_span in zip(actual_spans, predicted_spans):
    if actual_span == predicted_span:
        print("Actual: {}, Predicted: {}".format(actual_span, predicted_span))
        tp += 1
    else:
        fn += 1
        fp += 1

# Calculate precision, recall, and F1-score for each entity type
entity_types = set()
entity_tp = {}
entity_fp = {}
entity_fn = {}

for actual_span, predicted_span in zip(actual_spans, predicted_spans):
    for term, entity_type in actual_span:
        entity_types.add(entity_type)
        if actual_span == predicted_span:
            entity_tp[entity_type] = entity_tp.get(entity_type, 0) + 1
        else:
            entity_fn[entity_type] = entity_fn.get(entity_type, 0) + 1

for predicted_span in predicted_spans:
    for term, entity_type in predicted_span:
        if predicted_span not in actual_spans:
            entity_fp[entity_type] = entity_fp.get(entity_type, 0) + 1

# Calculate micro-average precision, recall, and F1-score
# micro_tp = sum(entity_tp.values())
# micro_fp = sum(entity_fp.values())
# micro_fn = sum(entity_fn.values())
micro_tp = tp
micro_fp = fp
micro_fn = fn

micro_precision = micro_tp / (micro_tp + micro_fp) if (micro_tp + micro_fp) > 0 else 0
micro_recall = micro_tp / (micro_tp + micro_fn) if (micro_tp + micro_fn) > 0 else 0
micro_f1_score = (2 * micro_precision * micro_recall) / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0

# Print evaluation results
print(f"TP: {tp}")
print(f"FP: {fp}")
print(f"FN: {fn}")
print()

for entity_type in entity_types:
    entity_precision = entity_tp.get(entity_type, 0) / (entity_tp.get(entity_type, 0) + entity_fp.get(entity_type, 0)) if (entity_tp.get(entity_type, 0) + entity_fp.get(entity_type, 0)) > 0 else 0
    entity_recall = entity_tp.get(entity_type, 0) / (entity_tp.get(entity_type, 0) + entity_fn.get(entity_type, 0)) if (entity_tp.get(entity_type, 0) + entity_fn.get(entity_type, 0)) > 0 else 0
    entity_f1_score = (2 * entity_precision * entity_recall) / (entity_precision + entity_recall) if (entity_precision + entity_recall) > 0 else 0

    print(f"Entity Type: {entity_type}")
    print(f"Precision: {entity_precision}")
    print(f"Recall: {entity_recall}")
    print(f"F1-Score: {entity_f1_score}")
    print()

print(f"Micro-Average Precision: {micro_precision}")
print(f"Micro-Average Recall: {micro_recall}")
print(f"Micro-Average F1-Score: {micro_f1_score}")

Actual: [('aircraft', 'COMPONENT')], Predicted: [('aircraft', 'COMPONENT')]
Actual: [('aircraft', 'COMPONENT')], Predicted: [('aircraft', 'COMPONENT')]
Actual: [('ActiveTrack', 'STATE')], Predicted: [('ActiveTrack', 'STATE')]
Actual: [('ActiveTrack', 'STATE')], Predicted: [('ActiveTrack', 'STATE')]
Actual: [('Aircraft', 'COMPONENT')], Predicted: [('Aircraft', 'COMPONENT')]
Actual: [('ActiveTrack', 'STATE')], Predicted: [('ActiveTrack', 'STATE')]
Actual: [('max', 'PARAMETER'), ('speed', 'PARAMETER')], Predicted: [('max', 'PARAMETER'), ('speed', 'PARAMETER')]
Actual: [('Obstacle', 'FUNCTION'), ('Avoidance', 'FUNCTION')], Predicted: [('Obstacle', 'FUNCTION'), ('Avoidance', 'FUNCTION')]
Actual: [('Aircraft', 'COMPONENT')], Predicted: [('Aircraft', 'COMPONENT')]
Actual: [('Aircraft', 'COMPONENT')], Predicted: [('Aircraft', 'COMPONENT')]
Actual: [('Max', 'PARAMETER'), ('altitude', 'PARAMETER')], Predicted: [('Max', 'PARAMETER'), ('altitude', 'PARAMETER')]
Actual: [('Aircraft', 'COMPONENT')],